In [1]:
import sys 
sys.path.append('./../src')

# my shit
from lib.finnhub.data import FinnhubData as finnhub
from lib.oscillators import macd_over_period, stochastic_over_period
from lib.charts import chart

In [16]:
ticker = 'F'

fh = finnhub()
data = fh.get_historical_data(ticker, fh.get_daily_closings, days = 180)

data.index = data['t']
data.sort_index()
data.drop_duplicates()

,t,o,l,h,c,v
t,,,,,,
2020-09-14 21:00:00,2020-09-14 21:00:00,7.12,7.12,7.13,7.13,4375
2020-09-15 21:00:00,2020-09-15 21:00:00,7.03,7.04,7.03,7.04,10573
2020-09-16 21:00:00,2020-09-16 21:00:00,7.03,7.04,7.05,7.07,21774
2020-09-17 21:00:00,2020-09-17 21:00:00,7.31,7.31,7.35,7.36,310143
2020-09-18 21:00:00,2020-09-18 21:00:00,7.21,7.23,7.22,7.23,23682
...,...,...,...,...,...,...
2021-03-04 22:00:00,2021-03-04 22:00:00,11.90,11.97,11.90,11.98,42126
2021-03-05 22:00:00,2021-03-05 22:00:00,12.11,12.25,12.29,12.30,329951
2021-03-08 22:00:00,2021-03-08 22:00:00,12.70,12.75,12.73,12.75,53958


In [6]:
days = 60

chart_data = list(data['c'].tail(days))

macd_vals = macd_over_period(list(data['c']), days)
stochastic_vals = stochastic_over_period(list(data['c']), days)

macd_line = list([i[0] for i in macd_vals])[-1-days:-1]
sig_line = list([i[1] for i in macd_vals])[-1-days:-1]
histo = list([i[2] for i in macd_vals])[-1-days:-1]

fastk_data = list([i[0] for i in stochastic_vals])
slowd_data = list([i[2] for i in stochastic_vals])

oscillator_dicts = [
    { 'chart': 'line', 'data': macd_line, 'label': 'macd' },
    { 'chart': 'line', 'data': sig_line, 'label': 'signal' },
    { 'chart': 'bar', 'data': histo, 'label': ''},
    { 'chart': 'line', 'data': fastk_data, 'label': 'fastk'},
    { 'chart': 'line', 'data': slowd_data, 'label': 'slowd'}
]

data_dicts = [
    { 'label': 'close', 'data': chart_data }
]

chart(ticker, data_dicts, oscillator_dicts)

ZeroDivisionError: float division by zero

In [14]:
def rsi(vals, period = 14):
    ep = 0.0000001
    rsis = []
    smoothed_rsis = []
    # get the rsi for the period days _before_ requested period 
    for i in range(0, period):
        range_vals = vals[-1-period*2-i:-1-period-i]
        deltas = list([(v[0]-v[1])/v[1] for v in range_vals])
        avg_gain = max(sum(list([abs(v) for v in deltas if v > 0])) / period, ep)
        avg_loss = max(sum(list([abs(v) for v in deltas if v < 0])) / period, ep)
        rsis = rsis + [100.0 - (100.0 / (avg_gain / avg_loss))]
    # use these rsis to calculate the actual request period
    for i in range(0, period):
        range_vals = vals[-1-period-i:-1-i]
        prev_rsis = rsis[-1-period-i:-1-i]
        delta = range_vals[-1][0] - range_vals[-1][1]
        if delta > 0:
            smoothed_gain = max(sum(prev_rsis + [delta]) / period, ep)
            smoothed_loss = max(sum(prev_rsis) / period, ep)
        else:
            smoothed_gain = max(sum(pref_rsis) / period, ep)
            smoothed_loss = max(sum(prev_rsis + [delta]) / period, ep)
    smoothed_rsis = smoothed_rsis + [100.0 - (100.0 / (smoothed_gain / smoothed_loss))]
    return smoothed_rsis[-1]

In [15]:
rsi_data = list(zip(data['c'], data['o']))
rsi(rsi_data, 20)

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'